## Implementing an LSTM RNN with Lasagne

This notebook aims to serve as an extra helper guide to accompany lstm_text_gen.py, included in the same directory. Here, we'll walk through the different key components and concepts involved. If you want to try it out, download a source text, call it `source.txt`, and then finally run `lstm_text_gen.py`. The outputs will be logged in a file called lstm.log

### Where do we even start?
When tasked with the challenge of implementing an LSTM RNN, a good place to start would be checking out what people have done in the past, and perhaps working from there. A great resource I found initially was [this implementation](https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py) of close to what we want in Keras. It's based off of an architecture described in this other amazing and fun blog post - [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). I would highly recommend checking both of these off.

Starting here will give us a good jumping off point with some existing pointers on how to structure the shape of the network, and what the inputs and outputs might look like. 

#### Inputs, Shape, and more!
Let's start by defining our goal here. We're trying to develop a character-level recurrent neural network using hidden layers composed of long short term memory elements, also known as LSTM. We'll be feeding in sequences of characters represented as one-hot vectors, and then using these inputs to predict the next character. The [Unreasonable Effectiveness](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) does a great job covering this concept. 

A quick example - 

With a sequence length of 5, an input and output might look like - 
INPUT: 'M', 'y', ' ', 'n', 'a'
OUTPUT: 'm'

Note: the major difference between this and a real implementation is in the one-hot encoding of each character.

What about shape? For most RNNs, there seems to be somewhat of a consensus around the internet that 2-3 hidden layers is appropriate. As for the size of each hidden layer, we'll be choosing 512 hidden nodes. 

### Data Processing and Representation
First, we'll need to get our data in the right shape. Let's take a look at the `gen_data` function defined in `lstm_text_gen.py`. 

In [ ]:
def generate_data(p, batch_size=BATCH_SIZE, data=in_text, return_target=True):
    '''
    Produces a semi-redundant batch of training samples from the location 'p' in the provided string (data).

    Inputs: 
        p (int): Position in the input text to start at
        batch_size (int): The number of frame shifts to make/training samples to make
        data (str): The input corpus
        return_target (boolean): Flag to determine whether to return the target vector

    Returns:
        x (np array): Input data matrix that takes the shape of (batch_size x SEQ_LENGTH x vocab_size)
        y (np array): Output data representing one-hot encoding of guessed character
    '''

    x = np.zeros((batch_size,SEQ_LENGTH,vocab_size))
    y = np.zeros(batch_size)

    for n in range(batch_size):
        ptr = n
        for i in range(SEQ_LENGTH):
            x[n,i,char_to_ix[data[p+ptr+i]]] = 1.
        if(return_target):
            y[n] = char_to_ix[data[p+ptr+SEQ_LENGTH]]
    return x, np.array(y,dtype='int32')

Let's walk through what's going on here. We take in an input text (represented as a string), and starting at an index `p` in the text, create `BATCH_SIZE` samples of length `SEQ_LENGTH`. Instead of the character itself, we choose the one-hot representation. Finally, the target is taken to be the next character past the end of the sequence. 

The result of all of this? A training matrix of size [batch_size, SEQ_LENGTH, vocab_size], and the corresponding target characters. Pretty neat!

### Building Up a Network

Now, let's go through the process of building up the network. We'll have four layers in total, which will be built with the python library Lasagne. In order, we'll have:

1. Input layer
2. LSTM layer 1
3. LSTM layer 2
4. Dense layer (output)

##### A quick aside on Lasagne
With Lasagne, creating a NN is as simple as compiling an input layer, middle (hidden layers) - of which there are many options, and the output. What’s great here is that Lasagne already has built in implementations for many different kinds of neural-net layers, from our barebones layer to a convolutional or LSTM layer.  From there, you define your cost function, optimizer (e.g. stochastic gradient descent), and train it up! Here are some good resources to get started with Lasagne -> [here](http://lasagne.readthedocs.org/en/latest/index.html), [here](https://github.com/Lasagne/Recipes/tree/master/tutorials), and finally [here](https://github.com/craffel/Lasagne-tutorial/blob/master/examples/tutorial.ipynb).

#### Input Layer
First, we'll define our input layer. By setting the first two dimensions as None, we are allowing them to vary. They correspond to batch size and sequence length, so we will be able to feed in batches of varying size with sequences of varying length.

In [ ]:
# Step 1 -> Build the input layer of the network. 
# Recurrent layers expect shape (batch size, SEQ_LENGTH, num_features)
l_in = lasagne.layers.InputLayer(shape=(None, None, vocab_size))

#### Recurrent Layers
Next, we'll define our two recurrent layers. 

We clip the gradients at GRAD_CLIP to prevent the problem of exploding gradients. I'll leave the specifics of the problems of exploding/disappearing gradients to the resources we've mentioned above.

In [ ]:
# Step 2 -> Build LSTM layer with takes l_in as input layer + clip gradient
l_forward_1 = lasagne.layers.LSTMLayer(
    l_in, N_HIDDEN, grad_clipping=GRAD_CLIP,
    nonlinearity=lasagne.nonlinearities.tanh)

l_forward_2 = lasagne.layers.LSTMLayer(
    l_forward_1, N_HIDDEN, grad_clipping=GRAD_CLIP,
    nonlinearity=lasagne.nonlinearities.tanh)

#### Output Layers
This is where things get a little complicated. The l_forward layer creates an output of dimension (batch_size, SEQ_LENGTH, N_HIDDEN). Since we are only interested in the final prediction, we isolate that quantity and feed it to the next layer. We can do this using a Lasagne SliceLayer.

In [ ]:
# Step 3 -> Feed the output from the recurrent LSTM layers into a traditional Dense Layer
# We only care about the final prediction here, so we isolate that and feed it into the next layer
# The output of the sliced layer will then be of size (batch_size, N_HIDDEN)
l_forward_slice = lasagne.layers.SliceLayer(l_forward_2, -1, 1)

Finally, we send the output from the SliceLayer through a traditional Dense layer using a softmax nonlinearity to get a probability distribution of the outputs. 

In [ ]:
# Step 4 -> Sliced output passed through softmax nonlinearity to create probability distribution of the prediction
# The output of this stage is (batch_size, vocab_size)
l_out = lasagne.layers.DenseLayer(l_forward_slice,
                                  num_units=vocab_size,
                                  W = lasagne.init.Normal(),
                                  nonlinearity=lasagne.nonlinearities.softmax)

#### Compilation and Cost

The last step before start training is to get the actual output from the network, create a cost expression to evaluate the 'goodness' of the network, and provide an update function to perform gradient descent. 

In [ ]:
# Theano tensor for the targets
target_values = T.ivector('target_output')

# lasagne.layers.get_output produces a variable for the output of the net
network_output = lasagne.layers.get_output(l_out)

# The loss function is calculated as the mean of the (categorical) cross-entropy between the prediction and target.
cost = T.nnet.categorical_crossentropy(network_output,target_values).mean()

# Retrieve all parameters from the network
all_params = lasagne.layers.get_all_params(l_out,trainable=True)

# Compute AdaGrad updates for training
logging.info("Computing updates ...")
updates = lasagne.updates.adagrad(cost, all_params, LEARNING_RATE)

# Theano functions for training and computing cost
logging.info("Compiling functions ...")
train = theano.function([l_in.input_var, target_values], cost, updates=updates, allow_input_downcast=True)
compute_cost = theano.function([l_in.input_var, target_values], cost, allow_input_downcast=True)

# Generate probabilities used to generate text from the network given the current state and seed text input
probs = theano.function([l_in.input_var],network_output,allow_input_downcast=True)

#### Generating Text
Let's write a function to generate text given the current state of the network. When we're training the network, we'll be calling this function periodically to get an idea for what the network is outputting. Let's walk through how this process is going to work.

We'll start with a seed generation text of at least length SEQ_LENGTH, from which we'll be building text. First, we'll take this and encode it to the one-hot representation we've been using. 

Then, we'll use the LSTM to predict the next character and store it in a dynamic list `sample_ix`. This character will be taken from the output using the `probs` function we defined above. We'll pick the one with the highest probability.

Finally, we construct a new sequence of characters using all but the first character of the original sequence and the character we just guessed. This becomes our new generation input.

In [ ]:
def generate_text(N=200):
        '''
        This function uses the user-provided string "generation_phrase" and current state of the RNN generate text.
        The function works in three steps:
        1. Convert string set in "generation_phrase" (which must be over SEQ_LENGTH characters long) 
           to encoded format. 
        2. Use LSTM to predict the next character and store it in a (dynamic) list sample_ix using the 'probs'
           function which was compiled above.
        3. Construct a new sequence using all but first characters of the provided string and the 
           predicted character. This sequence is then used to generate yet another character.
           
        This process continues for "N" characters. 
        '''

        assert(len(generation_phrase)>=SEQ_LENGTH)
        sample_ix = []
        x,_ = gen_data(len(generation_phrase)-SEQ_LENGTH, 1, generation_phrase,0)

        for i in range(N):
            # Pick the character that got assigned the highest probability
            ix = np.argmax(probs(x).ravel())
            sample_ix.append(ix)
            x[:,0:SEQ_LENGTH-1,:] = x[:,1:,:]
            x[:,SEQ_LENGTH-1,:] = 0
            x[0,SEQ_LENGTH-1,sample_ix[-1]] = 1. 

        random_snippet = generation_phrase + ''.join(ix_to_char[ix] for ix in sample_ix)    
        logging.info("----\n %s \n----" % random_snippet)

#### Training the Network

Almost there! The last step is to train the network. To train, we simply iteratively call our `train` function defined above and update the gradients the appropriate amount. At a calcualted interval, we print out the prediction of the network, in addition to the current loss.


In [ ]:
for it in xrange(data_size * num_epochs / BATCH_SIZE):
    generate_text() # Generate text using the p^th character as the start. 

    avg_cost = 0;
    for i in range(PRINT_FREQ):
        x,y = gen_data(p)

        p += SEQ_LENGTH + BATCH_SIZE - 1 
        if(p+BATCH_SIZE+SEQ_LENGTH >= data_size):
            logging.info('Carriage Return')
            p = 0;

        avg_cost += train(x, y)
    logging.info("Epoch {} average loss = {}".format(i*1.0*PRINT_FREQ/data_size*BATCH_SIZE, avg_cost / PRINT_FREQ))